# Sample Data Maker

this creates a sample data set that is in the format ready to be classified by a translator. thank you !

In [80]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
data = pd.read_json('../../data/output_5000_redo.json') 

In [3]:
data.head()

,link,linkWithDate,numWords,paragraphs,pdfName
0,https://hojishinbun.hoover.org/?a=d&d=jan19360...,https://hojishinbun.hoover.org/en/newspapers/j...,5,滅念祀迎獸轚鼴罾鐮 81 帝 1 世を黏レゎそれ —№ 11 にこ # は於 & しヤやニル...,jan_19360710_0019.pdf
1,https://hojishinbun.hoover.org/?a=d&d=kam19350...,https://hojishinbun.hoover.org/en/newspapers/k...,7,"「 ，！ 一 ： こ匿 - — 511. :.?-<-€¢€1—""--#118*141( -...",kam_19350811_0003.pdf
2,https://hojishinbun.hoover.org/?a=d&d=kam19420...,https://hojishinbun.hoover.org/en/newspapers/k...,5,一世二世の失業者は ’： 一 1^ ノ至急市協へ報吿せょ一 ：1 - ”£2” 日本人救濟の...,kam_19420128_0003.pdf
3,https://hojishinbun.hoover.org/?a=d&d=jan19410...,https://hojishinbun.hoover.org/en/newspapers/j...,6,號ニ十六百七千四万一第 當太和 3 声では開菜一一一却年 2.. ' と I て來る四 ：：...,jan_19410107_0005.pdf
4,https://hojishinbun.hoover.org/?a=d&d=kam19350...,https://hojishinbun.hoover.org/en/newspapers/k...,2,ジブラルタルとスュズ運河 ， 藤丼整 霉とな 〇 先が ^ ブチルタル線杯か 6111 8 ...,kam_19350922_0002.pdf


## Prep the Data to be sent for translation: 

Break up into the columns we want (link, numwords, paragraphs) and then add a label column that will be changed later by the translators.

In [16]:
data = data[data['numWords']>0]
data_cleaned = data[['link','numWords','paragraphs']]
# labels_col = np.zeros(int(data_cleaned.shape[0])) - 99 # -99 so we know if it was labelled yet 
# data_cleaned = data_cleaned.assign(**{'label': labels_col})

In [17]:
data_cleaned.sort_values('numWords', ascending = True)

,link,numWords,paragraphs
2746,https://hojishinbun.hoover.org/?a=d&d=tnw19310...,1,诚六廿 ^: 八午家箱 ？ 0 奶 日系市民の副檢事月偉千ドルを受く市民官吏中のタツブを切る...
2001,https://hojishinbun.hoover.org/?a=d&d=kam19320...,1,， 本學生にこの正義觀敎室で親日高唱非常識な排日敎師をやリこめる二 ， 蜜林某小學校の話 I...
2002,https://hojishinbun.hoover.org/?a=d&d=nws19360...,1,漦九十九对三第 閱斩ロ輞荩 # 餅 II 際 : こニ ^ 麵女一塲技競泳水會大オンリルべ一...
2003,https://hojishinbun.hoover.org/?a=d&d=nws19401...,1,( 电京卜 ： れは闻既じ去る九日御成年式を取げさせられた東久邇宮彰常 ： 土殿下にはよあふ...
2004,https://hojishinbun.hoover.org/?a=d&d=jan19290...,1,0 ¢8 ” み霣ニ供 ， 一 - 、 〇 六西故弗の鑛山女王自動米衝突 7 落命飲 31 ...
...,...,...,...
4166,https://hojishinbun.hoover.org/?a=d&d=nis19370...,21,. 一 「 爲讀港寄書 、 18111113118188— 自 E 『 ユ愿 446 戋 【...
4163,https://hojishinbun.hoover.org/?a=d&d=ksp19400...,23,為號八十七百七千一第報新哇加日一月一年五十和昭 たびあぢそく ^. 旅はよいもの味なもの ，...
4171,https://hojishinbun.hoover.org/?a=d&d=jan19380...,24,本嗍鼇に就ぃて其の寅默旳方法を猶ずる C 先き立つて先つ £ 鼠を二 0 に $ 黻し . ...
4195,https://hojishinbun.hoover.org/?a=d&d=tht19470...,25,"me & 厂 ・ ! ・ ー ー ー ー — 一 新日本の活舞臺に , 躍進の二世群像 … ..."


In [10]:
# sns.histplot(data = data_cleaned, x = 'numWords')

In [11]:
# let's create a data table that shows how many data points only have one hit: 
# data_cleaned.groupby('numWords').size()

In [14]:
# cleaning functions: 

def make_hit_arrays(string, hit, r):
    '''return a shortened version of STRING that is centered around HIT with R characters on each side of it
    STRING: any string
    HIT: any word
    returns:
    STRING wihout HIT, N: any positive integer that is the location of HIT in STRING. will return the first occurence of the first character of HIT'''
    ns = [] # this means there is no occurence of HIT if empty
    hit_strings = []
    
    if (hit in string): # first see that HIT is actually in STRING to avoid errors
        size_hit = len(hit) # how many characters to examine at once 
        
        for n in range(len(string) - size_hit): # itterate through STRING until you reach HIT
            
            if string[n:n+size_hit] == hit: # iterating till we reach HIT
                ns.append(n)
                hit_strings.append(string[max(0,n - r): min(n + r, len(string))])
    return hit_strings

In [143]:
def get_data_by_num_words(data, num_words = [1], num_samples =100):
    '''INPUT: Data: a data frame with the assumed column labls of link, numWords, paragraphs
        num_words: list-like of valid numWords in each data sample of the hit phrase
        num_samples: the number of data points you want back to categorize
    OUTPUT: a data frame that is ready to be translated'''
    
     # make an array of num_words 
    label_data = data[data['numWords'].isin(num_words)] # only keeps data that has the num words we want
    sample_table = label_data.sample(num_samples, random_state = 42) # sample the data randomly num_samples times 


    # add the label column: 
    labels_col = np.zeros(int(sample_table.shape[0])) - 99 # -99 so we know if it was labelled yet 
    data_final = sample_table.assign(**{'label': labels_col})

    return data_final
        
def split_data_by_num_words(data, hit = '二世', r = 25):
    '''given DATA with column numWords, split the data into rows of hit arrays 
    that share all column information except for paragraphs, which is now a split version, with numWords = 1. Then remove the original rows'''
    new_data_points = [] # an array of arrays that have the column values for each new data point 

    data_index = np.array([]) # an array that contains the original index value of the new data points 
    c = 0
    # for s in data['Paragraphs'].values:
    #     new_strings = make_hit_arrays(s, hit = '二世', r = r)
    #     data_index = np.append(data_index, np.zeros(len(new_strings)) + c)
    #     new_data_points.extend(new_strings)
    #     c += 1
    for s in data.values:
        hit_arrs = make_hit_arrays(s[2], hit = hit, r = r)
        for h in hit_arrs:
            new_data_points.append([s[0], 1, h]) # oiginal link, now only 1 numwords, new hit array, same label 
    
    new_data = pd.DataFrame(data = new_data_points, columns = data.columns)# the final data frame with cleaned arrays with hits at the center 
    return new_data

def get_data_by_years(data, years = ['1923','1924','1925'], num_samples = 100):
    '''do the same thing as getting numwords data but now do it by getting the year out from the link
    YEARS must be a string'''
    year_arr = data['link'].str.extract(pat = r'(19\d\d)')
    data_with_year = data.assign(**{'year':year_arr})
    data_valid = data_with_year[data_with_year['year'].isin(years)].sample(num_samples, random_state = 42)
    return data_valid.drop('year', axis = 1)

In [149]:
data_1920s = get_data_by_years(data_cleaned, years = ['1920','1921','1922','1923','1924','1925', '1926', '1927','1928', '1929'])

data_1920s_split = split_data_by_num_words(data_1920s)

In [81]:
# label_data = data_cleaned[data_cleaned['numWords'] == 1]
# label_data.head()

In [82]:
# sample_table = label_data.sample(100, random_state = 42)
# sample_indeces = sample_table.index
# sample_table = sample_table.drop('numWords', axis = 1).reset_index() # we don't need this since we know they all only have 1 and its training data

# # and then since we want to make sure we can keep the 
# # training data separate from the test data, we should take these data points out of the big data file, which is why we picked the random state we did
# sample_table.head(), sample_indeces

In [150]:
labels_arr = np.zeros(data_1920s_split.shape[0]) - 99

labels_arr

array([-99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -99., -99., -99., -99.,
       -99., -99., -99., -99., -99., -99., -99., -9

In [154]:
sample_table_1920s = data_1920s_split.assign(**{'label': labels_arr}).drop('numWords', axis = 1)

In [155]:
# # now to export!
from pathlib import Path  
filepath = Path('C:\\Users\\alica\\Documents\\URAP\\data\\training_data_sample2.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  # makes a directory for the CSV file to be written into 
sample_table_1920s.to_csv(filepath) 